# ArangoDB DGL Adapter Getting Started Guide  

<a href="https://colab.research.google.com/github/arangoml/dgl-adapter/blob/master/examples/ArangoDB_DGL_Adapter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![arangodb](https://raw.githubusercontent.com/arangoml/dgl-adapter/master/examples/assets/adb_logo.png)
<img src="https://raw.githubusercontent.com/arangoml/dgl-adapter/master/examples/assets/dgl_logo.png" width=40% />

Version: 1.0.0

Objective: Export Graphs from [ArangoDB](https://www.arangodb.com/), a multi-model Graph Database, into [Deep Graph Library](https://www.dgl.ai/) (DGL), a python package for graph neural networks, and vice-versa.

# Setup

In [ ]:
%%capture
!git clone -b oasis_connector --single-branch https://github.com/arangodb/interactive_tutorials.git
!git clone https://github.com/arangoml/dgl-adapter.git # !git clone -b 1.0.0 --single-branch https://github.com/arangoml/dgl-adapter.git
!rsync -av dgl-adapter/adbdgl_adapter/tests ./ --exclude=.git
!rsync -av interactive_tutorials/ ./ --exclude=.git
!pip3 install "git+https://github.com/arangoml/dgl-adapter.git#egg=adbdgl_adapter&subdirectory=adbdgl_adapter" # pip3 install adbdgl_adapter==1.0.0
!pip3 install matplotlib
!pip3 install pyArango
!pip3 install networkx ## For drawing purposes 

In [ ]:
import json
import oasis
import matplotlib.pyplot as plt

import torch
import networkx as nx
from dgl import remove_self_loop
from dgl.data import KarateClubDataset
from dgl.data import MiniGCDataset

from adbdgl_adapter.adbdgl_adapter import ArangoDB_DGL_Adapter
from adbdgl_adapter.adbdgl_controller import Base_ADBDGL_Controller

# Create a Temporary ArangoDB Instance

In [ ]:
# Request temporary instance from the managed ArangoDB Cloud Oasis.
con = oasis.getTempCredentials()

# Connect to the db via the python-arango driver
python_arango_db_driver = oasis.connect_python_arango(con)

# (Alternative) Connect to the db via the pyArango driver
# pyarango_db_driver = oasis.connect(con)[con['dbName']]

print()
print("https://{}:{}".format(con["hostname"], con["port"]))
print("Username: " + con["username"])
print("Password: " + con["password"])
print("Database: " + con["dbName"])

Feel free to use to above URL to checkout the UI!

# Data Import

We will use an Fraud Detection example graph, explained in more detail in this [interactive notebook](https://colab.research.google.com/github/joerg84/Graph_Powered_ML_Workshop/blob/master/Fraud_Detection.ipynb).

*Note the included arangorestore will only work on Linux system, if you want to run this notebook on a different OS please consider using the appropriate arangorestore from the [Download area](https://www.arangodb.com/download-major/).*

In [ ]:
!chmod -R 755 ./tools
!./tools/arangorestore -c none --server.endpoint http+ssl://{con["hostname"]}:{con["port"]} --server.username {con["username"]} --server.database {con["dbName"]} --server.password {con["password"]} --default-replication-factor 3  --input-directory "tests/data/fraud_dump"

# Create Graph

The graph we will be using in the following looks as follows:

![networkX](https://github.com/arangoml/networkx-adapter/blob/master/examples/assets/fraud_graph.jpeg?raw=1) 

In [ ]:
edge_definitions = [
    {
        "edge_collection": "accountHolder",
        "from_vertex_collections": ["customer"],
        "to_vertex_collections": ["account"],
    },
    {
        "edge_collection": "transaction",
        "from_vertex_collections": ["account"],
        "to_vertex_collections": ["account"],
    },
]

name = "fraud-detection"
python_arango_db_driver.delete_graph(name, ignore_missing=True)
fraud_graph = python_arango_db_driver.create_graph(name, edge_definitions=edge_definitions)

print("Graph Setup done.")
print(fraud_graph)

Feel free to visit the ArangoDB UI using the above link and login data and check the Graph!

# Create Adapter

Connect the ArangoDB_DGL_Adapter to our temp ArangoDB cluster:

In [ ]:
adbdgl_adapter = ArangoDB_DGL_Adapter(con)

# ArangoDB to DGL



## Via ArangoDB Graph

In [ ]:
# Define graph name
graph_name = "fraud-detection"

# Create DGL graph from ArangoDB graph
dgl_g = adbdgl_adapter.arangodb_graph_to_dgl(graph_name)

# You can also provide valid Python-Arango AQL query options to the command above, like such:
# dgl_g = aadbdgl_adapter.arangodb_graph_to_dgl(graph_name, ttl=1000, stream=True)
# See more here: https://docs.python-arango.com/en/main/specs.html#arango.aql.AQL.execute

# Show graph data
print(dgl_g)
print(dgl_g.ndata)
print(dgl_g.edata)

## Via ArangoDB Collections

In [ ]:
# Define collection
vertex_collections = {"account", "Class", "customer"}
edge_collections = {"accountHolder", "Relationship", "transaction"}

# Create DGL from ArangoDB collections
dgl_g = adbdgl_adapter.arangodb_collections_to_dgl("fraud-detection", vertex_collections, edge_collections)

# You can also provide valid Python-Arango AQL query options to the command above, like such:
# dgl_g = adbdgl_adapter.arangodb_collections_to_dgl("fraud-detection", vertex_collections, edge_collections, ttl=1000, stream=True)
# See more here: https://docs.python-arango.com/en/main/specs.html#arango.aql.AQL.execute

# Show graph data
print(dgl_g)
print(dgl_g.ndata)
print(dgl_g.edata)

## Via ArangoDB Metagraph

In [ ]:
# Define Metagraph
fraud_detection_metagraph = {
    "vertexCollections": {
        "account": {"rank"},
        "Class": {"concrete", "name"},
        "customer": {"Sex", "Ssn", "rank"},
    },
    "edgeCollections": {
        "accountHolder": {},
        "Relationship": {},
        "transaction": {"receiver_bank_id", "sender_bank_id", "transaction_amt", "transaction_date", "trans_time"},
    },
}

# Introduce the new controller class
class FraudDetection_ADBDGL_Controller(Base_ADBDGL_Controller):
    """ArangoDB-DGL controller.

    Responsible for controlling how ArangoDB attributes
    are converted into DGL features, and vice-versa.
    """

    def _adb_attribute_to_dgl_feature(self, key: str, col: str, val):
        """
        Given an ArangoDB attribute key, its assigned value (for an arbitrary document),
        and the collection it belongs to, convert it to a valid
        DGL feature: https://docs.dgl.ai/en/0.6.x/guide/graph-feature.html.

        NOTE: DGL only accepts 'attributes' (a.k.a features) of numerical types.
        """
        if type(val) in [int, float, bool]:
          return val

        if col == "transaction":
          if key == "transaction_date":
            return int(str(val).replace("-", ""))
  
          if key == "trans_time":
            return int(str(val).replace(":", ""))
  
        if col == "customer":
          if key == "Sex":
            return 0 if val == "M" else 1

          if key == "Ssn":
            return int(str(val).replace("-", ""))

        if col == "Class":
          if key == "name":
            if val == "Bank":
              return 0
            elif val == "Branch":
              return 1
            elif val == "Account":
              return 2
            elif val == "Customer":
              return 3
            else:
              return -1

fraud_adbgl_adapter = ArangoDB_DGL_Adapter(con, FraudDetection_ADBDGL_Controller)

# Create DGL Graph from attributes
dgl_g = fraud_adbgl_adapter.arangodb_to_dgl('FraudDetection',  fraud_detection_metagraph)

# You can also provide valid Python-Arango AQL query options to the command above, like such:
# dgl_g = adbdgl_adapter.arangodb_to_dgl(graph_name = 'FraudDetection',  fraud_detection_metagraph, ttl=1000, stream=True)
# See more here: https://docs.python-arango.com/en/main/specs.html#arango.aql.AQL.execute

# Show graph data
print('\n--------------')
print(dgl_g)
print('\n--------------')
print(dgl_g.ndata)
print('--------------\n')
print(dgl_g.edata)

# DGL to ArangoDB

## Example 1: DGL Karate Graph

In [ ]:
# Load the dgl graph & draw
dgl_karate_graph = KarateClubDataset()[0]
nx.draw(dgl_karate_graph.to_networkx(), with_labels=True)

# Create the ArangoDB graph
name = "Karate"
python_arango_db_driver.delete_graph(name, drop_collections=True, ignore_missing=True)
adb_karate_graph = adbdgl_adapter.dgl_to_arangodb(name, dgl_karate_graph)


print(f"\nInspect the graph here: https://tutorials.arangodb.cloud:8529/_db/{con['dbName']}/_admin/aardvark/index.html#graph/{name}\n")

print("https://{}:{}".format(con["hostname"], con["port"]))
print("Username: " + con["username"])
print("Password: " + con["password"])
print("Database: " + con["dbName"])

print(f"\nView the original graph below:")

## Example 2: DGL MiniGCDataset Graphs

In [ ]:
# Load the dgl graphs & draw
dgl_lollipop_graph = remove_self_loop(MiniGCDataset(8, 7, 8)[3][0])
dgl_lollipop_graph.ndata["random_ndata"] = torch.rand(dgl_lollipop_graph.num_nodes())
dgl_lollipop_graph.edata["random_edata"] = torch.rand(dgl_lollipop_graph.num_edges())
plt.figure(1)
nx.draw(dgl_lollipop_graph.to_networkx(), with_labels=True)

dgl_hypercube_graph = remove_self_loop(MiniGCDataset(8, 8, 9)[4][0])
dgl_hypercube_graph.ndata["random_ndata"] = torch.rand(dgl_hypercube_graph.num_nodes())
dgl_hypercube_graph.edata["random_edata"] = torch.rand(dgl_hypercube_graph.num_edges())
plt.figure(2)
nx.draw(dgl_hypercube_graph.to_networkx(), with_labels=True)

dgl_clique_graph = remove_self_loop(MiniGCDataset(8, 6, 7)[6][0])
dgl_clique_graph.ndata["random_ndata"] = torch.rand(dgl_clique_graph.num_nodes())
dgl_clique_graph.edata["random_edata"] = torch.rand(dgl_clique_graph.num_edges())
plt.figure(3)
nx.draw(dgl_clique_graph.to_networkx(), with_labels=True)

# Create the ArangoDB graphs
lollipop = "Lollipop"
hypercube = "Hypercube"
clique = "Clique"

python_arango_db_driver.delete_graph(lollipop, drop_collections=True, ignore_missing=True)
python_arango_db_driver.delete_graph(hypercube, drop_collections=True, ignore_missing=True)
python_arango_db_driver.delete_graph(clique, drop_collections=True, ignore_missing=True)

adb_lollipop_graph = adbdgl_adapter.dgl_to_arangodb(lollipop, dgl_lollipop_graph)
adb_hypercube_graph = adbdgl_adapter.dgl_to_arangodb(hypercube, dgl_hypercube_graph)
adb_clique_graph = adbdgl_adapter.dgl_to_arangodb(clique, dgl_clique_graph)

print("\nInspect the graphs here:\n")
print(f"1) https://tutorials.arangodb.cloud:8529/_db/{con['dbName']}/_admin/aardvark/index.html#graph/{lollipop}")
print(f"2) https://tutorials.arangodb.cloud:8529/_db/{con['dbName']}/_admin/aardvark/index.html#graph/{hypercube}")
print(f"3) https://tutorials.arangodb.cloud:8529/_db/{con['dbName']}/_admin/aardvark/index.html#graph/{clique}\n")

print("https://{}:{}".format(con["hostname"], con["port"]))
print("Username: " + con["username"])
print("Password: " + con["password"])
print("Database: " + con["dbName"])

print(f"\nView the original graphs below:")